In [17]:
import numpy as np
import cv2 as cv
import sys
import os
from pathlib import Path
from typing import Dict, Tuple
from numpy.testing import assert_allclose, assert_almost_equal
np.set_printoptions(precision=4)

In [29]:
def euroc_stereo_cam_configuration() -> Tuple[Dict, Dict]:
    """
    Defines the stereo cam configuration of EUROC setup.
    """

    image_shape = (752, 480)
    # instrinsic parameters
    K_l = np.array([[458.654, 0.0, 367.215], 
                    [0.0, 457.296, 248.375], 
                    [0.0, 0.0, 1.0],
    ])

    K_r = np.array([[457.587, 0.0, 379.999],
                    [0.0, 456.134, 255.238],
                    [0.0, 0.0, 1]
    ])
    # transformations 
    T_B_C0 = np.array([
        [0.0148655429818, -0.999880929698, 0.00414029679422, -0.0216401454975],
        [0.999557249008, 0.0149672133247, 0.025715529948, -0.064676986768],
        [-0.0257744366974, 0.00375618835797, 0.999660727178, 0.00981073058949],
        [ 0.0, 0.0, 0.0, 1.0]
    ])

    T_B_C1 = np.array([
        [0.0125552670891, -0.999755099723, 0.0182237714554, -0.0198435579556],
        [0.999598781151, 0.0130119051815, 0.0251588363115, 0.0453689425024],
        [-0.0253898008918, 0.0179005838253, 0.999517347078, 0.00786212447038],
        [0.0, 0.0, 0.0, 1.0]
    ])

    # Rotation parameters
    R_l = np.array([
         [0.999966347530033, -0.001422739138722922, 0.008079580483432283], 
         [0.001365741834644127, 0.9999741760894847, 0.007055629199258132], 
        [-0.008089410156878961, -0.007044357138835809, 0.9999424675829176],
    ])

    R_r = np.array([
         [0.9999633526194376, -0.003625811871560086, 0.007755443660172947], 
         [0.003680398547259526, 0.9999684752771629, -0.007035845251224894], 
        [-0.007729688520722713, 0.007064130529506649, 0.999945173484644],
    ])

    # distortion parameter
    # 
    d_l = np.array([-0.28340811, 0.07395907, 0.00019359, 1.76187114e-05, 0.0])
    d_r = np.array([-0.28368365, 0.07451284, -0.00010473, -3.555907e-05, 0.0])

    # Projection
    P_l = np.array([[435.2046959714599, 0, 367.4517211914062, 0],  
                    [0, 435.2046959714599, 252.2008514404297, 0],  
                    [0, 0, 1, 0]
    ])

    P_r = np.array([[435.2046959714599, 0, 367.4517211914062, -47.90639384423901],
                    [ 0, 435.2046959714599, 252.2008514404297, 0], 
                    [0, 0, 1, 0]])
    cam_L = {'T' : T_B_C0, 'k': K_l, 'd': d_l, 'R': R_l, 'P': P_l}
    cam_R = {'T' : T_B_C1, 'k': K_r, 'd': d_r, 'R': R_r, 'P': P_r}
    return cam_L, cam_R

def rectification_map(cam_L:Dict, cam_R: Dict, image_shape) -> Tuple[np.ndarray, np.ndarray]:
    """
    """
    map_l = cv.initUndistortRectifyMap(cam_L['k'], cam_L['d'], cam_L['R'], cam_L['P'], image_shape, cv.CV_32F)
    map_r = cv.initUndistortRectifyMap(cam_R['k'], cam_R['d'], cam_R['R'], cam_R['P'], image_shape, cv.CV_32F)
    print(f"Shape of undistort map in stereo camera: {map_l.shape}")
    return map_l, map_r 

In [24]:
cam_l, cam_r = euroc_stereo_cam_configuration()
print(f'left camera = {cam_l}', sep=" ")
print(f'right camera = {cam_r}')

left camera = {'T': array([[ 0.0149, -0.9999,  0.0041, -0.0216],
       [ 0.9996,  0.015 ,  0.0257, -0.0647],
       [-0.0258,  0.0038,  0.9997,  0.0098],
       [ 0.    ,  0.    ,  0.    ,  1.    ]]), 'k': array([[458.654,   0.   , 367.215],
       [  0.   , 457.296, 248.375],
       [  0.   ,   0.   ,   1.   ]]), 'd': array([-2.8341e-01,  7.3959e-02,  1.9359e-04,  1.7619e-05,  0.0000e+00]), 'R': array([[ 1.    , -0.0014,  0.0081],
       [ 0.0014,  1.    ,  0.0071],
       [-0.0081, -0.007 ,  0.9999]]), 'P': array([[435.2047,   0.    , 367.4517,   0.    ],
       [  0.    , 435.2047, 252.2009,   0.    ],
       [  0.    ,   0.    ,   1.    ,   0.    ]])}
right camera = {'T': array([[ 0.0126, -0.9998,  0.0182, -0.0198],
       [ 0.9996,  0.013 ,  0.0252,  0.0454],
       [-0.0254,  0.0179,  0.9995,  0.0079],
       [ 0.    ,  0.    ,  0.    ,  1.    ]]), 'k': array([[457.587,   0.   , 379.999],
       [  0.   , 456.134, 255.238],
       [  0.   ,   0.   ,   1.   ]]), 'd': array([-2.83

In [40]:
R = cam_l['R'].T @ cam_r['R']
T_lr = np.linalg.inv(cam_l['T']) @ cam_r['T'] 
# The translation vector is in the frame of left camera.
# so the left camera is relatively an identity matrix here.
t = T_lr[:3, 3]

print(f'Transformation matrix right wrt left : \n { T_lr}')
print(f'Rotation matrix matrix right wrt left : \n { T_lr[:3, :3]}, \n Translation between two cameras: \n {t.transpose()}')
print(f'Rotation matrix R : \n {R}')
print(f'How close are the rotation matrix: \n{np.allclose(R, T_lr[:3, :3])}')

Transformation matrix right wrt left : 
 [[ 1.0000e+00 -2.3171e-03 -3.4339e-04  1.1007e-01]
 [ 2.3121e-03  9.9990e-01 -1.4091e-02 -1.5661e-04]
 [ 3.7601e-04  1.4090e-02  9.9990e-01  8.8938e-04]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
Rotation matrix matrix right wrt left : 
 [[ 1.0000e+00 -2.3171e-03 -3.4339e-04]
 [ 2.3121e-03  9.9990e-01 -1.4091e-02]
 [ 3.7601e-04  1.4090e-02  9.9990e-01]], 
 Translation between two cameras: 
 [ 0.1101 -0.0002  0.0009]
Rotation matrix R : 
 [[ 1.0000e+00 -2.3171e-03 -3.4339e-04]
 [ 2.3121e-03  9.9990e-01 -1.4091e-02]
 [ 3.7601e-04  1.4090e-02  9.9990e-01]]
How close are the rotation matrix: 
True


In [42]:
P = R@cam_r['k']
print(f'rotated calibration matrix - right camera: \n {P}')
# We already have the Rotation and translation of the stereo camera given to us.
# So we have not calculated the Essential matrix by matching points and then recovered pose from it.
# We need to form the rectification matrix R_rect
# https://www.cs.cmu.edu/~16385/s17/Slides/13.1_Stereo_Rectification.pdf
# Get the epipole
e = t/np.linalg.norm(t)
# direction vector of optical axis
# Now we are in the camera frame
z_cam = np.array([0, 0, 1])
r1 = e
r2 = 

rotated calibration matrix - right camera: 
 [[ 4.5759e+02 -1.0569e+00  3.7941e+02]
 [ 1.0580e+00  4.5609e+02  2.5608e+02]
 [ 1.7206e-01  6.4269e+00  4.7390e+00]]
